In [116]:
import importlib
import Integrate_ContentRC
import Integrate_SurpriseCollaborativeRC
import Integrate_DeepLearningCollaborativeRC
import numpy as np
importlib.reload(Integrate_ContentRC)
importlib.reload(Integrate_SurpriseCollaborativeRC)
importlib.reload(Integrate_DeepLearningCollaborativeRC)


from Integrate_DeepLearningCollaborativeRC import EmbeddingNetwork, DeepLearningCollaborativeBasedRecommender
from Integrate_SurpriseCollaborativeRC import SurpriseCollaborativeBasedRecommender
from Integrate_ContentRC import ContentBasedEnum
from Integrate_ContentRC import ContentBasedRecommender
from typing import List, Tuple

np.random.seed(0)

In [117]:
datasetPathReviews: str = "reviews_filtered.csv"
useCols: List[str] = ["RecipeId", "AuthorId", "Rating"]
targetAlign: List[str] = ["AuthorId", "RecipeId", "Rating"]
ratingScale: Tuple = (1, 5)
measures: List[str] = ["RMSE"]
weights: List[float] = [0.24, 0.41]

In [118]:
column_of_interest = ["Name", "Description", "RecipeCategory",
                        "TotalTime", "AuthorName", "RecipeInstructions",
                        "Keywords"]
datasetPath: str = "recipes_filtered.csv"
glove_file: str = "Embeddings/glove.6B.50d.txt"    
modelName: str = "all-MiniLM-L12-v2"

In [119]:
DLCollaborative = DeepLearningCollaborativeBasedRecommender(datasetPathReviews, useCols)
DLCollaborative.alignDataset(targetAlign)
DLCollaborative.createDataset()
DLCollaborative.splitDataset(0.1)
DLCollaborative.setupNetwork(150)
modelDict, history, lrHistory = DLCollaborative.trainNetwork(epochs=5)
DLCollaborative.modelRMSE("cpu")
rec = DLCollaborative.getRecommendation(412, 5)

/mnt/Development-Main/Xor/githubProjects/python/sisrek/tubes/Integrate_DeepLearningCollaborativeRC.py:115: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  torch.nn.init.xavier_uniform(layer.weight)


Loss getting better on epoch 1
[001/005] train: 1.7875 - val: 2.5388
Loss getting better on epoch 2
[002/005] train: 1.4556 - val: 2.2893
Loss getting better on epoch 3
[003/005] train: 1.4426 - val: 2.2274
Loss getting better on epoch 4
[004/005] train: 1.3439 - val: 2.2028
[005/005] train: 1.3106 - val: 2.2326
Final RMSE: 0.0973
tensor([412, 412, 412,  ..., 412, 412, 412])
0         0
1         1
2         2
3         3
4         4
       ... 
4005    696
4006    696
4007    696
4008    696
4009    696
Name: newRecipeId, Length: 4010, dtype: int64


In [121]:
rec.sort_values("pred", ascending=False)

,recipeId,pred
755,56,4.788732
1513,1041,4.753612
2106,1292,4.736423
1404,632,4.734728
1339,943,4.727441
...,...,...
643,7,3.951718
1070,752,3.928298
2385,1549,3.921664
1422,939,3.914929


In [14]:
DLCollaborative.df.iloc[502, :]

AuthorId       8602
RecipeId         83
Rating            5
newAuthorId     412
newRecipeId     389
Name: 502, dtype: int64

In [134]:
surpriseCF = SurpriseCollaborativeBasedRecommender(datasetPathReviews, ratingScale,
                                                    useCols, measures)
surpriseCF.alignDataset(targetAlign)
surpriseCF.splitDataset()
surpriseCF.fit(0.5, 0.5)
surpriseCF.getRecommendations(412, 5)


Optimizer Weights [0.86784429 0.17221314]
Optimizer Weights [0.18636341 0.19741862]


([Prediction(uid=412, iid=5090, r_ui=None, est=4.639861378157691, details={'was_impossible': False}),
  Prediction(uid=412, iid=9869, r_ui=None, est=4.637928692731506, details={'was_impossible': False}),
  Prediction(uid=412, iid=10404, r_ui=None, est=4.530287878982761, details={'was_impossible': False}),
  Prediction(uid=412, iid=13021, r_ui=None, est=4.520343018229543, details={'was_impossible': False}),
  Prediction(uid=412, iid=4384, r_ui=None, est=4.470554279726398, details={'was_impossible': False})],
 [Prediction(uid=412, iid=4627, r_ui=None, est=5, details={'was_impossible': False}),
  Prediction(uid=412, iid=8679, r_ui=None, est=5, details={'was_impossible': False}),
  Prediction(uid=412, iid=8627, r_ui=None, est=5, details={'was_impossible': False}),
  Prediction(uid=412, iid=5090, r_ui=None, est=5, details={'was_impossible': False}),
  Prediction(uid=412, iid=3288, r_ui=None, est=5, details={'was_impossible': False})])

In [2]:
contentBased = ContentBasedRecommender(datasetPath, column_of_interest)
contentBased.preprocess_dataset()
contentBased.prepareRecipeIndices("Name")
contentBased.setup(glove_file, modelName)

/mnt/Development-Main/Xor/githubProjects/python/sisrek/tubes/Integrate_ContentRC.py:46: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[]' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  self.df.fillna("[]", inplace=True)
/mnt/Dev_partition/anaconda3/envs/universal/lib/python3.9/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Setup Completed


In [3]:
contentBased.fit("contents", "contents_metadata", )
contentBased.calculate_similarity()

Similarity matrix is generated!
(9529, 9529)
(9529, 9529)
(9529, 9529)
(9529, 9529)


In [6]:
contentBased.get_recommendation("twinkie del mar", 10, ContentBasedEnum.PRE_Content)

[1, 2962, 2785, 8735, 89, 441, 7489, 1937, 7056, 8661]